In [ ]:
## setup
## TODO: move all under bird2022wheels
!pip install ../input/birds-inference-pip-wheels/torchaudio-0.8.1-cp37-cp37m-manylinux1_x86_64.whl ../input/birds-inference-pip-wheels/torch-1.8.1-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/birds-inference-pip-wheels/audiomentations-0.16.0-py3-none-any.whl --no-index --no-deps
!pip install ../input/birds-inference-pip-wheels/torchlibrosa-0.0.9-py3-none-any.whl --no-index --no-deps
!pip install ../input/birds2022wheels/nnAudio-0.3.1-py3-none-any.whl
!cp -r ../input/timmlatest ../working/timmlatest
!pip install -U ../working/timmlatest
!rm -rf ../working/timmlatest

In [1]:
## https://github.com/Selimonder/birdclef2022/
import os, sys, glob, math

os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

github_folder = "."
sys.path.append(github_folder)

In [36]:
import cv2
import gc
import re
import torch
import librosa
import argparse, warnings

import numpy as np
import pandas as pd
import IPython.display as ipd

import zoo
from training.config import load_config
from training.datasets import BirdDatasetOOF


cv2.ocl.setUseOpenCL(False)
cv2.setNumThreads(0)

from tqdm import tqdm
from torch.utils.data import DataLoader

warnings.simplefilter("ignore")

## models

In [61]:
def load_model(conf_path: str, weights_path: str, prefix: str, suffix: str, fold: int, to_device: bool = True, is_test:bool = False):
    conf = load_config(conf_path)
    conf['encoder_params']['pretrained'] = False
    
    snapshot_name = "{}{}_{}_{}_{}".format(prefix, conf["network"], conf["encoder_params"]["encoder"], fold, suffix)
    weights_dir = weights_path
    weights_path = os.path.join(weights_path, snapshot_name)
    
    def test_model():
        model_names = ['SED', 'TimmClassifier_v1', 'TimmClassifier_v2', 'TimmClassifier2021', 'TimmClassifier',
                       'SEDTrainableFFT', 'C1C2', 'TimmClassifierSplitCrop_v1']
        print(model_names)
        snapshot_name = "pretrain_TimmClassifier_eca_nfnet_l0_0_last"#"pretrained_eca_nfnet_l0.pth"# "pretrain_TimmClassifier_eca_nfnet_l0_0_last"
        weights_path = os.path.join(weights_dir, snapshot_name)   
        
        raw_conf_model = conf["network"]
        for model_name in model_names:
            try:
                print("\n\n => current model: ", model_name)
                conf["network"] = model_name
                load_model(weights_path)
            except Exception as e:
                print(f"model not found", e)
                continue
            print("success!")
        conf["network"] = raw_conf_model
    
    def load_model(weights_path):
        print(weights_path)
        model = zoo.__dict__[conf["network"]](**conf["encoder_params"])
        model = torch.nn.DataParallel(model).cuda()
        print("=> loading checkpoint '{}''".format(weights_path))
        checkpoint = torch.load(weights_path, map_location="cpu")
        print("epoch", checkpoint["epoch"])
        model.load_state_dict(checkpoint["state_dict"])
        model.eval()
        if to_device: model.cuda()
        return model
    
    if is_test:
        test_model()
    else:
        return load_model(weights_path)


def load_checkpoint(model: torch.nn.Module, weight_path):
    checkpoint_path = weight_path
    if not checkpoint_path:
        return
    if os.path.isfile(checkpoint_path):
        print("=> loading checkpoint '{}'".format(checkpoint_path))
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
        print(checkpoint.keys())
        state_dict_name = "state_dict"
        if state_dict_name not in checkpoint:
            state_dict_name = "model"
        if state_dict_name in checkpoint:
            conflicts = 0
            state_dict = checkpoint[state_dict_name]
            # print("state_dict:", state_dict.keys())
            # state_dict = {re.sub("^module.", "", k): w for k, w in state_dict.items()}
            # print("state_dict:", state_dict.keys())
            orig_state_dict = model.state_dict()
            # print("orig_state_dict", orig_state_dict.keys())
            mismatched_keys = []
            for k, v in state_dict.items():
                ori_size = orig_state_dict[k].size() if k in orig_state_dict else None
                if v.size() != ori_size:
                    conflicts += 1
                    # print("SKIPPING!!! Shape of {} changed from {} to {}".format(k, v.size(), ori_size))
                    mismatched_keys.append(k)
            for k in mismatched_keys:
                del state_dict[k]
            model.load_state_dict(state_dict, strict=False)
            print("=> Not loaded checkpoint '{}' (epoch {})"
                  .format(checkpoint_path, checkpoint['epoch']))
        else:
            model.load_state_dict(checkpoint)
    else:
        print("=> no checkpoint found at '{}'".format(checkpoint_path))
    print("conflicts: ", conflicts)
    return conflicts
    
def load_model_v1(conf_path: str, weights_path: str, prefix: str, suffix: str, fold: int, to_device: bool = True, is_test:bool = False):
    conf = load_config(conf_path)
    conf['encoder_params']['pretrained'] = False
    
    snapshot_name = "{}{}_{}_{}_{}".format(prefix, conf["network"], conf["encoder_params"]["encoder"], fold, suffix)
    weights_dir = weights_path
    weights_path = os.path.join(weights_path, snapshot_name)
    
    def test_model():
        model_names = ['SED', 'TimmClassifier_v1', 'TimmClassifier_v2', 'TimmClassifier2021', 'TimmClassifier',
                       'SEDTrainableFFT', 'C1C2', 'TimmClassifierSplitCrop_v1']
        print(model_names)
        snapshot_name = "pretrained_eca_nfnet_l0.pth" # "pretrain_TimmClassifier_eca_nfnet_l0_0_last"#"pretrained_eca_nfnet_l0.pth"# "pretrain_TimmClassifier_eca_nfnet_l0_0_last"
        weights_path = os.path.join(weights_dir, snapshot_name)   
        
        raw_conf_model = conf["network"]
        mx, res = 1e5, ""
        for model_name in model_names:
            try:
                print("\n\n => current model: ", model_name)
                conf["network"] = model_name
                c = load_model(weights_path)
                if c < mx:
                    mx = c
                    res = model_name
            except Exception as e:
                print(f"model not found", e)
                continue
            print("success!")
        conf["network"] = raw_conf_model
        print("\n=> best fit:", res, " conflicts:", mx)
    
    def load_model(weights_path):
        print(weights_path)
        model = zoo.__dict__[conf["network"]](**conf["encoder_params"])
        model = torch.nn.DataParallel(model).cuda()
        return load_checkpoint(model, weights_path)
    
    if is_test:
        test_model()
    else:
        return load_model(weights_path)








In [62]:
model = load_model_v1(conf_path  = f"{github_folder}/configs/cls_nf0_v5.json",
                               weights_path = f"{github_folder}/weights",
                               prefix       = "nf0_v5_",
                               suffix       = sx,
                               fold         = i,
                               is_test=True)
# models.append(model)

['SED', 'TimmClassifier_v1', 'TimmClassifier_v2', 'TimmClassifier2021', 'TimmClassifier', 'SEDTrainableFFT', 'C1C2', 'TimmClassifierSplitCrop_v1']


 => current model:  SED
./weights/pretrained_eca_nfnet_l0.pth
initing SED model...
model not found NormFreeNet.__init__() got an unexpected keyword argument 'duration'


 => current model:  TimmClassifier_v1
./weights/pretrained_eca_nfnet_l0.pth
initing CLS features model 15 duration...
pretrained model...
{'in_chans': 1, 'drop_path_rate': 0.2, 'drop_rate': 0.5}
=> loading checkpoint './weights/pretrained_eca_nfnet_l0.pth'
odict_keys(['stem.conv1.weight', 'stem.conv1.bias', 'stem.conv1.gain', 'stem.conv2.weight', 'stem.conv2.bias', 'stem.conv2.gain', 'stem.conv3.weight', 'stem.conv3.bias', 'stem.conv3.gain', 'stem.conv4.weight', 'stem.conv4.bias', 'stem.conv4.gain', 'stages.0.0.downsample.conv.weight', 'stages.0.0.downsample.conv.bias', 'stages.0.0.downsample.conv.gain', 'stages.0.0.conv1.weight', 'stages.0.0.conv1.bias', 'stages.0.0.conv1

pretrained model...
{'in_chans': 1, 'drop_path_rate': 0.2, 'drop_rate': 0.5}
=> loading checkpoint './weights/pretrained_eca_nfnet_l0.pth'
odict_keys(['stem.conv1.weight', 'stem.conv1.bias', 'stem.conv1.gain', 'stem.conv2.weight', 'stem.conv2.bias', 'stem.conv2.gain', 'stem.conv3.weight', 'stem.conv3.bias', 'stem.conv3.gain', 'stem.conv4.weight', 'stem.conv4.bias', 'stem.conv4.gain', 'stages.0.0.downsample.conv.weight', 'stages.0.0.downsample.conv.bias', 'stages.0.0.downsample.conv.gain', 'stages.0.0.conv1.weight', 'stages.0.0.conv1.bias', 'stages.0.0.conv1.gain', 'stages.0.0.conv2.weight', 'stages.0.0.conv2.bias', 'stages.0.0.conv2.gain', 'stages.0.0.conv2b.weight', 'stages.0.0.conv2b.bias', 'stages.0.0.conv2b.gain', 'stages.0.0.conv3.weight', 'stages.0.0.conv3.bias', 'stages.0.0.conv3.gain', 'stages.0.0.attn_last.conv.weight', 'stages.1.0.downsample.conv.weight', 'stages.1.0.downsample.conv.bias', 'stages.1.0.downsample.conv.gain', 'stages.1.0.conv1.weight', 'stages.1.0.conv1.bias', 

=> loading checkpoint './weights/pretrained_eca_nfnet_l0.pth'
odict_keys(['stem.conv1.weight', 'stem.conv1.bias', 'stem.conv1.gain', 'stem.conv2.weight', 'stem.conv2.bias', 'stem.conv2.gain', 'stem.conv3.weight', 'stem.conv3.bias', 'stem.conv3.gain', 'stem.conv4.weight', 'stem.conv4.bias', 'stem.conv4.gain', 'stages.0.0.downsample.conv.weight', 'stages.0.0.downsample.conv.bias', 'stages.0.0.downsample.conv.gain', 'stages.0.0.conv1.weight', 'stages.0.0.conv1.bias', 'stages.0.0.conv1.gain', 'stages.0.0.conv2.weight', 'stages.0.0.conv2.bias', 'stages.0.0.conv2.gain', 'stages.0.0.conv2b.weight', 'stages.0.0.conv2b.bias', 'stages.0.0.conv2b.gain', 'stages.0.0.conv3.weight', 'stages.0.0.conv3.bias', 'stages.0.0.conv3.gain', 'stages.0.0.attn_last.conv.weight', 'stages.1.0.downsample.conv.weight', 'stages.1.0.downsample.conv.bias', 'stages.1.0.downsample.conv.gain', 'stages.1.0.conv1.weight', 'stages.1.0.conv1.bias', 'stages.1.0.conv1.gain', 'stages.1.0.conv2.weight', 'stages.1.0.conv2.bias', 

=> loading checkpoint './weights/pretrained_eca_nfnet_l0.pth'
odict_keys(['stem.conv1.weight', 'stem.conv1.bias', 'stem.conv1.gain', 'stem.conv2.weight', 'stem.conv2.bias', 'stem.conv2.gain', 'stem.conv3.weight', 'stem.conv3.bias', 'stem.conv3.gain', 'stem.conv4.weight', 'stem.conv4.bias', 'stem.conv4.gain', 'stages.0.0.downsample.conv.weight', 'stages.0.0.downsample.conv.bias', 'stages.0.0.downsample.conv.gain', 'stages.0.0.conv1.weight', 'stages.0.0.conv1.bias', 'stages.0.0.conv1.gain', 'stages.0.0.conv2.weight', 'stages.0.0.conv2.bias', 'stages.0.0.conv2.gain', 'stages.0.0.conv2b.weight', 'stages.0.0.conv2b.bias', 'stages.0.0.conv2b.gain', 'stages.0.0.conv3.weight', 'stages.0.0.conv3.bias', 'stages.0.0.conv3.gain', 'stages.0.0.attn_last.conv.weight', 'stages.1.0.downsample.conv.weight', 'stages.1.0.downsample.conv.bias', 'stages.1.0.downsample.conv.gain', 'stages.1.0.conv1.weight', 'stages.1.0.conv1.bias', 'stages.1.0.conv1.gain', 'stages.1.0.conv2.weight', 'stages.1.0.conv2.bias', 

In [ ]:
models = []

## best only for now
suffixes = ["lb"]#, "f1_score" "last"]
folds    = 5

for i in range(folds):
    for sx in suffixes:
        try:
            model = load_model(conf_path    = f"{github_folder}/configs/cls_nf0_v5.json",
                               weights_path = f"{github_folder}/weights/nfnet-baseline-bs16",
                               prefix       = "nf0_v5_",
                               suffix       = sx,
                               fold         = i)
            models.append(model)
        except Exception as e:
            print(f"model not found", e)

['SED', 'TimmClassifier_v1', 'TimmClassifier_v2', 'TimmClassifier2021', 'TimmClassifier', 'SEDTrainableFFT', 'C1C2', 'TimmClassifierSplitCrop_v1']


 => current model:  SED
./weights/pretrained_eca_nfnet_l0.pth
initing SED model...


 => current model:  TimmClassifier_v1
./weights/pretrained_eca_nfnet_l0.pth
initing CLS features model 15 duration...
pretrained model...
{'in_chans': 1, 'drop_path_rate': 0.2, 'drop_rate': 0.5}
=> loading checkpoint './weights/pretrained_eca_nfnet_l0.pth''


 => current model:  TimmClassifier_v2
./weights/pretrained_eca_nfnet_l0.pth
initing CLS features model 15 duration...
pretrained model...
{'in_chans': 1, 'drop_path_rate': 0.2, 'drop_rate': 0.5}
=> loading checkpoint './weights/pretrained_eca_nfnet_l0.pth''


 => current model:  TimmClassifier2021
./weights/pretrained_eca_nfnet_l0.pth
initing CLS features model 15 duration...
pretrained model...
{'in_chans': 1, 'drop_path_rate': 0.2, 'drop_rate': 0.5}
=> loading checkpoint './weights/pretrained_eca_nf

In [31]:
len(models)

5

## infer

In [32]:
CLASSES = "abethr1 abhori1 abythr1 afbfly1 afdfly1 afecuc1 affeag1 afgfly1 afghor1 afmdov1 afpfly1 afpkin1 afpwag1 afrgos1 afrgrp1 afrjac1 afrthr1 amesun2 augbuz1 bagwea1 barswa bawhor2 bawman1 bcbeat1 beasun2 bkctch1 bkfruw1 blacra1 blacuc1 blakit1 blaplo1 blbpuf2 blcapa2 blfbus1 blhgon1 blhher1 blksaw1 blnmou1 blnwea1 bltapa1 bltbar1 bltori1 blwlap1 brcale1 brcsta1 brctch1 brcwea1 brican1 brobab1 broman1 brosun1 brrwhe3 brtcha1 brubru1 brwwar1 bswdov1 btweye2 bubwar2 butapa1 cabgre1 carcha1 carwoo1 categr ccbeat1 chespa1 chewea1 chibat1 chtapa3 chucis1 cibwar1 cohmar1 colsun2 combul2 combuz1 comsan crefra2 crheag1 crohor1 darbar1 darter3 didcuc1 dotbar1 dutdov1 easmog1 eaywag1 edcsun3 egygoo equaka1 eswdov1 eubeat1 fatrav1 fatwid1 fislov1 fotdro5 gabgos2 gargan gbesta1 gnbcam2 gnhsun1 gobbun1 gobsta5 gobwea1 golher1 grbcam1 grccra1 grecor greegr grewoo2 grwpyt1 gryapa1 grywrw1 gybfis1 gycwar3 gyhbus1 gyhkin1 gyhneg1 gyhspa1 gytbar1 hadibi1 hamerk1 hartur1 helgui hipbab1 hoopoe huncis1 hunsun2 joygre1 kerspa2 klacuc1 kvbsun1 laudov1 lawgol lesmaw1 lessts1 libeat1 litegr litswi1 litwea1 loceag1 lotcor1 lotlap1 luebus1 mabeat1 macshr1 malkin1 marsto1 marsun2 mcptit1 meypar1 moccha1 mouwag1 ndcsun2 nobfly1 norbro1 norcro1 norfis1 norpuf1 nubwoo1 pabspa1 palfly2 palpri1 piecro1 piekin1 pitwhy purgre2 pygbat1 quailf1 ratcis1 raybar1 rbsrob1 rebfir2 rebhor1 reboxp1 reccor reccuc1 reedov1 refbar2 refcro1 reftin1 refwar2 rehblu1 rehwea1 reisee2 rerswa1 rewsta1 rindov rocmar2 rostur1 ruegls1 rufcha2 sacibi2 sccsun2 scrcha1 scthon1 shesta1 sichor1 sincis1 slbgre1 slcbou1 sltnig1 sobfly1 somgre1 somtit4 soucit1 soufis1 spemou2 spepig1 spewea1 spfbar1 spfwea1 spmthr1 spwlap1 squher1 strher strsee1 stusta1 subbus1 supsta1 tacsun1 tafpri1 tamdov1 thrnig1 trobou1 varsun2 vibsta2 vilwea1 vimwea1 walsta1 wbgbir1 wbrcha2 wbswea1 wfbeat1 whbcan1 whbcou1 whbcro2 whbtit5 whbwea1 whbwhe3 whcpri2 whctur2 wheslf1 whhsaw1 whihel1 whrshr1 witswa1 wlwwar wookin1 woosan wtbeat1 yebapa1 yebbar1 yebduc1 yebere1 yebgre1 yebsto1 yeccan1 yefcan yelbis1 yenspu1 yertin1 yesbar1 yespet1 yetgre1 yewgre1".split()
N_CLASS = len(CLASSES)

In [33]:

def prepare_clip(fpath, frame_length, sample_rate):
    """
    Prepare audio clip for inference
    """
    infer_frame_length = frame_length
    batch = {"wav_tensors": [], "end_times": []}
    
    waveform, sample_rate = librosa.load(fpath, sr=sample_rate, mono=True)
    n_parts = math.ceil(len(waveform) / int(infer_frame_length * sample_rate))
    
    for seg_idx in range(n_parts): 
        end_time = (seg_idx + 1) * frame_length
        seg_wav  = waveform[(end_time*sample_rate)-(sample_rate*frame_length):end_time*sample_rate]
        
        wav_tensor = torch.from_numpy(seg_wav)
        
        if len(wav_tensor) == frame_length * sample_rate:
            batch["wav_tensors"].append(wav_tensor.unsqueeze(0))
        elif len(wav_tensor) < frame_length * sample_rate:
            wav_tensor = torch.nn.functional.pad(wav_tensor, (0, (frame_length * sample_rate) - len(wav_tensor)))
            batch["wav_tensors"].append(wav_tensor.unsqueeze(0))
        elif len(wav_tensor) > frame_length * sample_rate:
            wav_tensor = wav_tensor[:(frame_length * sample_rate)]
            batch["wav_tensors"].append(wav_tensor.unsqueeze(0))
            
        batch["end_times"].append(end_time)
    batch["wav_tensors"] = torch.stack(batch["wav_tensors"]).cuda()
    return batch, n_parts, len(waveform)

@torch.no_grad()
def predict_clip(models, batch, n_parts, frame_length):
    preds = np.zeros([len(models), n_parts, N_CLASS])
    for m_idx, model in enumerate(models):
        with torch.cuda.amp.autocast():
            preds[m_idx] = model(batch["wav_tensors"], is_test=True)["logit"].sigmoid().cpu().numpy()
    return preds.mean(0) ## max by model    
    

In [34]:
len(models)

5

In [35]:
%%time


@torch.no_grad()
def generate_preds():
    debug = False
    preds        = []
    test_files   = sorted(glob.glob("/kaggle/input/birdclef-2023/test_soundscapes/*.ogg")) 
    sample_rate  = 32000
    
    scored_birds = np.array(CLASSES)

    frame_length = 5
    infer_frame_length = 5

    print("test file num:", len(test_files))
    for fpath in test_files:
        file_id = os.path.basename(fpath).replace(".ogg", "")
        batch, n_parts, len_waveform = prepare_clip(fpath, frame_length, sample_rate)
        clip_preds = predict_clip(models, batch, n_parts, frame_length)
        preds.append(clip_preds)
        
        ## switch to infer frame length
        n_parts_sub = math.ceil(len_waveform / int(infer_frame_length * sample_rate))
        clip_preds = np.array_split(clip_preds, n_parts_sub, axis=0)
        
                
    prob_array = np.array(preds)
    
    scored_bird_name2idx = {}
    for i, x in enumerate(scored_birds):
        scored_bird_name2idx[x] = i

    return prob_array, scored_bird_name2idx

prob_array, scored_bird_name2idx = generate_preds()
# del models
# gc.collect()
# torch.cuda.empty_cache()

test file num: 1
CPU times: user 695 ms, sys: 11.8 ms, total: 707 ms
Wall time: 705 ms


In [36]:
prob_array.shape

(1, 120, 264)

----

In [37]:
# %cd /kaggle/input/birdnetgit

In [38]:
# import os
# import sys
# import json
# import math
# import glob
# import librosa
# import operator
# import argparse
# import datetime
# import traceback

# from multiprocessing import Pool, freeze_support

# import numpy as np

# import config as cfg
# import audio
# import model
# from analyze import *

# import pandas as pd
# import numpy as np

In [39]:
# df = pd.read_csv("../birdclef-2022/eBird_Taxonomy_v2021.csv")
# s2id = lambda x: df[(df['PRIMARY_COM_NAME'] == x)].SPECIES_CODE.tolist()[0]

# cfg.MODEL_PATH = '/kaggle/input/birdnetgit/checkpoints/V2.1/BirdNET_GLOBAL_2K_V2.1_Model_FP32.tflite'
# cfg.MDATA_MODEL_PATH = '/kaggle/input/birdnetgit/checkpoints/V2.1/BirdNET_GLOBAL_2K_V2.1_MData_Model_FP32.tflite'
# cfg.LABELS_FILE = '/kaggle/input/birdnetgit/checkpoints/V2.1/BirdNET_GLOBAL_2K_V2.1_Labels.txt'
# cfg.TRANSLATED_LABELS_PATH = '/kaggle/input/birdnetgit/labels/V2.1'

# cfg.SIG_LENGTH = 5
# cfg.SIG_OVERLAP = 0 
# cfg.SIG_MINLEN = 5 

# # Load eBird codes, labels

# def loadCodes(CODEC_FILE):

#     with open(CODEC_FILE, 'r') as cfile:
#         codes = json.load(cfile)

#     return codes

# cfg.CODES = loadCodes("./eBird_taxonomy_codes_2021E.json")
# def loadLabels(labels_file):

#     labels = []
#     with open(labels_file, 'r') as lfile:
#         for line in lfile.readlines():
#             labels.append(line.replace('\n', ''))    

#     return labels

# def prepare_clip(fpath):
#     """
#     Prepare audio clip for inference
#     """
#     sample_rate = cfg.SAMPLE_RATE
#     frame_length = cfg.SIG_LENGTH
#     infer_frame_length = frame_length
    
#     sample_rate = 48000
#     chunks = []
    
#     waveform, sample_rate = librosa.load(fpath, sr=sample_rate, mono=True, res_type='kaiser_fast')
#     n_parts = math.ceil(len(waveform) / int(infer_frame_length * sample_rate))
    
#     for seg_idx in range(n_parts): 
#         end_time = (seg_idx + 1) * frame_length
#         chunk  = waveform[(end_time*sample_rate)-(sample_rate*frame_length):end_time*sample_rate]
                
#         if len(chunk) == frame_length * sample_rate:
#             chunks.append(chunk)
#         elif len(chunk) < frame_length * sample_rate:
#             chunk = np.pad(chunk, (0, (frame_length * sample_rate) - len(chunk)))
#             chunks.append(chunk)
#         elif len(chunk) > frame_length * sample_rate:
#             chunk = chunk[:(frame_length * sample_rate)]
#             chunks.append(chunk)
            
#     return chunks


# scored_birds = np.array(['akiapo', 'aniani', 'apapan', 'barpet', 'crehon', 'elepai', 'ercfra',
#                           'hawama', 'hawcre', 'hawgoo', 'hawhaw', 'hawpet1', 'houfin', 'iiwi',
#                           'jabwar', 'maupar', 'omao', 'puaioh', 'skylar', 'warwhe1', 'yefcan'])

# cfg.LABELS = loadLabels(cfg.LABELS_FILE)

# model.loadModel()



In [40]:
# bird_dict = {}
# for i, lb in enumerate(cfg.LABELS):
#     try:
#         lbb = s2id(lb.split("_")[-1])
#         if lbb in scored_birds:
#             bird_dict[lbb] = i
#     except Exception as e:
#         continue
    
# bird_dict["aniani"] = 0

# sorted_bd = dict(sorted(bird_dict.items(), key=lambda item: item[0]))
# get_idx = list(sorted_bd.values())

In [41]:
# # def predict_single(test_path)

# def generate_preds_bn():
#     test_files   = sorted(glob.glob("/kaggle/input/birdclef-2022/test_soundscapes/*.ogg"))
    
#     preds = []
#     file_ids = []
    
#     for fpath in test_files:
#         chunks = prepare_clip(fpath)
#         pred = predict(chunks)[:, get_idx]
#         preds.append(pred)
#         file_id = os.path.basename(fpath).replace(".ogg", "")
#         file_ids.append(file_id)
        
#     preds = np.array(preds)
#     return preds, file_ids
        

In [42]:
# %%time
# preds_bn, file_ids = generate_preds_bn()


In [43]:
# preds_bn[:, :, 1] = prob_array[:, :, 1] # 有一种鸟类缺失

In [44]:
def generate_file_ids():
    test_files   = sorted(glob.glob("/kaggle/input/birdclef-2022/test_soundscapes/*.ogg"))
    
    file_ids = []
    
    for fpath in test_files:
        file_id = os.path.basename(fpath).replace(".ogg", "")
        file_ids.append(file_id)
        
    return file_ids

file_ids = generate_file_ids()

In [45]:
## fix aniani
avg_preds = prob_array

avg_preds.shape

(1, 120, 264)

In [46]:
submission   = []
infer_frame_length = 5
threshold = 0.2

for clip_preds, file_id in zip(avg_preds, file_ids):
    for frame_idx, pred in enumerate(clip_preds):
        end_time = (frame_idx + 1) * infer_frame_length
        d = {
            "row_id": f"{file_id}_{end_time}",
            # "target": pred[bi] > threshold,
        }
        for bi, bird in enumerate(CLASSES):
            d[bird] = pred[bi]
        submission.append(d)

In [47]:
%cd /kaggle/working

[Errno 2] No such file or directory: '/kaggle/working'
/home/mobius/code/kaggle/BLEF/rank1/birdclef-2022


In [48]:
df_submission = pd.DataFrame(submission).set_index("row_id")
df_submission.to_csv("submission.csv")

In [49]:
# print(f"""
# total rows : {len(df_submission)}
# activated  : {len(df_submission[(df_submission.target==True)])}
# """)

df_submission.head()

,abethr1,abhori1,abythr1,afbfly1,afdfly1,afecuc1,affeag1,afgfly1,afghor1,afmdov1,...,yebsto1,yeccan1,yefcan,yelbis1,yenspu1,yertin1,yesbar1,yespet1,yetgre1,yewgre1
row_id,,,,,,,,,,,,,,,,,,,,,
soundscape_453028782_5,0.126487,0.082135,0.080978,0.048578,0.050510,0.101477,0.075433,0.024035,0.160303,0.053009,...,0.014767,0.017149,0.048940,0.027983,0.046173,0.084363,0.033920,0.033705,0.054019,0.053470
soundscape_453028782_10,0.066574,0.164496,0.030779,0.066138,0.037714,0.032803,0.077673,0.018512,0.038596,0.144080,...,0.008452,0.019594,0.077687,0.045750,0.048686,0.069257,0.020712,0.027779,0.021687,0.075800
soundscape_453028782_15,0.080323,0.081409,0.044128,0.017276,0.023266,0.057581,0.047525,0.018962,0.037262,0.075879,...,0.003203,0.015982,0.083878,0.041879,0.019554,0.062750,0.014851,0.031625,0.027172,0.034436
soundscape_453028782_20,0.032924,0.054861,0.033537,0.064783,0.026036,0.041327,0.038520,0.048108,0.087000,0.055145,...,0.003781,0.016900,0.036923,0.051416,0.037178,0.045248,0.030559,0.035767,0.048044,0.034192
soundscape_453028782_25,0.032156,0.053903,0.038099,0.042058,0.025344,0.041344,0.033179,0.035039,0.080115,0.055972,...,0.003029,0.013768,0.044425,0.037059,0.028831,0.037836,0.025044,0.025127,0.041965,0.029175
